<a href="https://colab.research.google.com/github/ragul-n/Gender-bias-in-Indian-cinema/blob/master/dictionary_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ragul-n/socialsent.git
!unzip "/content/socialsent.zip"

Cloning into 'socialsent'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 230 (delta 28), reused 26 (delta 16), pack-reused 182
Receiving objects: 100% (230/230), 2.62 MiB | 6.61 MiB/s, done.
Resolving deltas: 100% (103/103), done.
unzip:  cannot find or open /content/socialsent.zip, /content/socialsent.zip.zip or /content/socialsent.zip.ZIP.


In [2]:
!pip install theano

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 7.1 MB/s 
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668112 sha256=6eafe150d7b3c8f5388faae0a15b12641d40dde7385ba83c5272b9234cfc95ee
  Stored in directory: /root/.cache/pip/wheels/84/cb/19/235b5b10d89b4621f685112f8762681570a9fa14dc1ce904d9
Successfully built theano


In [193]:
import numpy as np
import pandas as pd
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [194]:
from gensim.models import KeyedVectors

word_vectors= KeyedVectors.load("/content/drive/MyDrive/Dataset/word2vec/subtitles_word_emb_v6.word_vectors")



In [195]:
from socialsent.socialsent.representations.embedding import Embedding

word_vector_pairs = dict({})
for idx, key in enumerate(word_vectors.vocab):
    word_vector_pairs[key] = word_vectors.get_vector(key)

word_vector_pairs=pd.Series(word_vector_pairs)

word_embeddings=Embedding(np.stack(word_vector_pairs.values), word_vector_pairs.index)

In [84]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


27602

In [196]:
%autoreload 2
from socialsent.socialsent.polarity_induction_methods import densify

polarity=densify(word_embeddings, 
                 positive_seeds=["career","profession", "corpor", "salari", "offici", "busi" ],  
                 negative_seeds=["home", "child", "famili", "cousin", "marriag", "wed"])
        


Preparing to learn embedding tranformation
Learning embedding transformation
Epoch: 1 Loss: -0.011975004337728024
Epoch: 2 Loss: -0.018415210768580437
Epoch: 3 Loss: -0.02420518919825554
Epoch: 4 Loss: -0.029281074181199074
Epoch: 5 Loss: -0.03363789990544319


In [197]:
polarity=pd.Series(polarity)
polarity=(polarity-polarity.mean())/polarity.std()

In [200]:
high_polarity=polarity[(polarity>2) + (polarity<-2)].sort_values()

In [208]:
print("Top 20 Positive Words:\n",high_polarity[-20:])
print()
print("Top 20 Negative Words:\n", high_polarity[:20])

Top 20 Positive Words:
 noida            3.142402
ba               3.146924
commiss          3.158805
margin           3.165213
illeg            3.166429
multin           3.169315
mnc              3.233893
firm             3.241313
traine           3.246798
dean             3.276607
infrastructur    3.298714
branch           3.319574
gurgaon          3.395784
mart             3.401303
hr               3.451258
busi             3.577356
contractor       3.618554
govern           3.763551
blacklist        3.820889
corpor           4.394174
dtype: float32

Top 20 Negative Words:
 wed           -3.665559
courtesan     -3.623161
bridal        -3.448468
home          -3.399797
child         -3.363264
heirloom      -3.353174
groom         -3.336725
dishonour     -3.321970
bvndl         -3.279756
baptiz        -3.223227
pardhu        -3.195765
fukri         -3.177620
marriag       -3.158957
paramour      -3.157662
dishonor      -3.155554
zubeida       -3.154561
mahalaya      -3.139421
haimavat

In [ ]:
!zip -r "/content/socialsent.zip" "/content/socialsent"